In [1]:
import os, json
from dotenv import load_dotenv
import boto3

In [2]:
print(boto3.__version__)

1.40.2


In [3]:
# Load AWS credentials from .env file
load_dotenv()

True

In [4]:
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_session_token = os.getenv("AWS_SESSION_TOKEN")

In [5]:
# Set the AWS region (change as needed, e.g., 'us-east-1')
aws_region = 'us-east-1'

### Setup AWS Clients and Model IDs

In [6]:

import boto3
import json

region = "us-west-2"  
job_kb_id = "9PFZZ5FEIF"  
job_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

### LLM Query Analyzer Node (Intent Classification)

In [7]:
def llm_query_analyzer(question, model_id=job_model_id, region=region):
    """
    Uses LLM to classify the user query into one or more agent tool intents.
    Returns a list of detected intents (e.g., ['job'], ['courses'], etc.).
    """
    prompt = f"""
                Classify the user's query intent(s) as one or more of the following: job, courses, mentoring, objectives.
                Respond ONLY with a comma-separated list of intents (e.g., "job", "courses").
                User query:
                {question}
              """
    client = boto3.client("bedrock-runtime", region_name=region)
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 10,
        "temperature": 0.0
    }
    response = client.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(body)
    )
    categories = json.loads(response['body'].read())['content'][0]['text']
    # Parse and clean result
    return [cat.strip().lower() for cat in categories.split(",") if cat.strip()]

### Courses KB Tool

In [8]:
class CoursesKBTool:
    def __init__(self, kb_id, region, model_id):
        self.kb_id = kb_id
        self.region = region
        self.model_id = model_id
        self.kb_client = boto3.client('bedrock-agent-runtime', region_name=region)
        self.llm_client = boto3.client('bedrock-runtime', region_name=region)

    def retrieve_context(self, query_text, extra_context=None):
        """
        Retrieves relevant course documents from KB using collaborative filtering hints
        (e.g., if extra_context has skills from a job, use that to filter courses)
        """
        # Collaborative filter: combine user query with job-related skills if present
        effective_query = query_text
        if extra_context:
            effective_query += "\n" + extra_context

        response = self.kb_client.retrieve(
            knowledgeBaseId=self.kb_id,
            retrievalQuery={'text': effective_query}
        )
        return [
            f"[{i+1}] {doc.get('content', {}).get('text', 'No content')}"
            for i, doc in enumerate(response.get('retrievalResults', []))
        ]

    def ask_llm(self, context, question):
        prompt = f"""
You are a helpful learning advisor who only recommends courses using the context below.
If the user's question or skill requirement is not addressed by the provided courses, reply honestly and, if possible, summarize the closest match.

Context (LinkedIn/Absorb courses):
{context}

User question:
{question}

Please cite which courses you are referencing.
"""
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 512,
            "temperature": 0.1
        }
        response = self.llm_client.invoke_model(
            modelId=self.model_id,
            contentType="application/json",
            accept="application/json",
            body=json.dumps(body)
        )
        return json.loads(response['body'].read())['content'][0]['text']

    def run(self, question, extra_context=None):
        context_list = self.retrieve_context(question, extra_context=extra_context)
        context = "\n".join(context_list) if context_list else "[No context found]"
        return self.ask_llm(context, question)

### Job Recommendation Tool

In [9]:
class JobKBTool:
    def __init__(self, kb_id, region, model_id):
        self.kb_id = kb_id
        self.region = region
        self.model_id = model_id
        self.kb_client = boto3.client('bedrock-agent-runtime', region_name=region)
        self.llm_client = boto3.client('bedrock-runtime', region_name=region)

    def retrieve_context(self, query_text):
        response = self.kb_client.retrieve(
            knowledgeBaseId=self.kb_id,
            retrievalQuery={'text': query_text}
        )
        return [
            f"[{i+1}] {doc.get('content', {}).get('text', 'No content')}"
            for i, doc in enumerate(response.get('retrievalResults', []))
        ]

    def ask_llm(self, context, question):
        prompt = f"""
You are a helpful assistant that only answers using the provided job postings context below.
If the user's question is not addressed by any of the provided postings, reply honestly that you did not find relevant results, and summarize any closest match if possible.

Context:
{context}

User question:
{question}

Please cite which postings you are referencing.
"""
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 512,
            "temperature": 0.1
        }
        response = self.llm_client.invoke_model(
            modelId=self.model_id,
            contentType="application/json",
            accept="application/json",
            body=json.dumps(body)
        )
        return json.loads(response['body'].read())['content'][0]['text']

    def run(self, question):
        # Step 1: Retrieve job postings context from KB
        context_list = self.retrieve_context(question)
        
        if not context_list:
            # No results at all → direct to portal
            return f"I couldn't find any job postings related to your request. Please check our internal job portal: https://www.smartrecruiters.com/app/employee-portal/5fa27e743f1d3276c0088aaa/jobs"
        
        context = "\n".join(context_list)

        # Step 2: Ask LLM to return exact or closest match
        prompt = f"""
    You are a job recommendation assistant.
    User is asking: "{question}"

    You have the following job postings:
    {context}

    Instructions:
    1. If a job exactly matches the role in the user's query, return it.
    2. If there is no exact match, return the closest technical/developer-related roles.
    3. If no relevant matches are found at all, say: "I couldn't find any job postings related to your request. Please check our internal job portal: https://www.smartrecruiters.com/app/employee-portal/5fa27e743f1d3276c0088aaa/jobs"
    4. Do not invent job postings — only use what's in the context.
    """
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 512,
            "temperature": 0.1
        }
        response = self.llm_client.invoke_model(
            modelId=self.model_id,
            contentType="application/json",
            accept="application/json",
            body=json.dumps(body)
        )
        return json.loads(response['body'].read())['content'][0]['text']

### Agent Tool Dispatcher Node

In [10]:
def agent_tool_dispatcher(intents, question, tool_map):
    """
    Calls the correct tool(s) for each intent.
    If both 'job' and 'courses' are in the intents, extract skills from the job tool
    and use as collaborative filtering context for the courses tool.
    """
    results = {}
    job_tool = tool_map.get("job")
    courses_tool = tool_map.get("courses")

    job_answer = None
    skills_context = None

    if "job" in intents and job_tool:
        # Step 1: Get job posting context
        job_answer = job_tool.run(question)
        results["job"] = job_answer

        # Try to extract skills from job context for collaborative course recommendation
        # For now, let's simply use the question itself or a placeholder
        # (In production, use NER or another LLM call to extract skill phrases)
        skills_context = question  # Replace with actual skills extraction logic if available

    if "courses" in intents and courses_tool:
        # Use collaborative filtering by passing skills_context to the Courses tool
        courses_answer = courses_tool.run(question, extra_context=skills_context)
        results["courses"] = courses_answer

    # You can add similar logic for mentoring/objectives as more KBs come online
    return results


In [11]:
def global_reflexion_node(all_tool_answers, user_query, model_id, region):
    """
    Aggregates answers from all tools and refines them into one cumulative, human-like answer.
    """
    aggregated_context = "\n\n".join(
        f"{intent.title()} Tool:\n{answer}" for intent, answer in all_tool_answers.items()
    )
    prompt = f"""
You are an assistant that synthesizes answers from multiple tools (Job KB, Courses KB, etc).
Your task is to provide a single, clear, and honest response to the user's question by combining the outputs below.

If there are no relevant job postings, say so transparently and summarize any related postings if needed.
If there are relevant courses, recommend them with context.
If neither is found, explain honestly.
Do not repeat the same information or cite the same item twice.

User question:
{user_query}

Aggregated tool answers:
{aggregated_context}

Final response:
"""
    client = boto3.client("bedrock-runtime", region_name=region)
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 512,
        "temperature": 0.1
    }
    response = client.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(body)
    )
    return json.loads(response['body'].read())['content'][0]['text']

### Main Pipeline

In [21]:
# Instantiate the job and courses tools
job_tool = JobKBTool(
    kb_id="9PFZZ5FEIF",
    region="us-west-2",
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)
courses_tool = CoursesKBTool(
    kb_id="DENPFPR7CR",
    region="us-west-2",
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)

tool_map = {
    "job": job_tool,
    "courses": courses_tool
    # Add more tools as needed
}

def run_agentic_workflow(question):
    print(f"User Query: {question}")
    intents = llm_query_analyzer(question)
    print("Detected Intents:", intents)
    answers = agent_tool_dispatcher(intents, question, tool_map)
    final_answer = global_reflexion_node(
        all_tool_answers=answers,
        user_query=question,
        model_id=job_model_id,   # You can use your preferred LLM
        region=region
    )
    print("\n===== Cumulative Agent Answer =====\n")
    print(final_answer)
    return final_answer

# Example multi-intent query
user_query = "Are there any sales jobs"
run_agentic_workflow(user_query)

User Query: Are there any sales jobs
Detected Intents: ['job']

===== Cumulative Agent Answer =====

# Available Sales Job Opportunities

Yes, there are several sales positions currently available across different locations and industries:

## Sales Executive Roles
- **Sales Executive - Informa Markets Latam** (Mexico City)
  - B2B sales position with hybrid work arrangement
  - Focuses on client acquisition and relationship management
  - Requires a Bachelor's degree, B2B sales experience, and English proficiency

- **Sales Executive - Informa Markets** (Hangzhou, China)
  - Retail industry position focused on the CBME exhibition
  - Involves client development and meeting sales targets
  - Requires a Bachelor's degree and at least 1 year of sales experience

- **Sales Executive** (Bangkok, Thailand) - Taylor and Francis
  - Home-based position with travel requirements
  - Promotes online resources to academic markets
  - Requires 3+ years sales experience and strong English skills

#

"# Available Sales Job Opportunities\n\nYes, there are several sales positions currently available across different locations and industries:\n\n## Sales Executive Roles\n- **Sales Executive - Informa Markets Latam** (Mexico City)\n  - B2B sales position with hybrid work arrangement\n  - Focuses on client acquisition and relationship management\n  - Requires a Bachelor's degree, B2B sales experience, and English proficiency\n\n- **Sales Executive - Informa Markets** (Hangzhou, China)\n  - Retail industry position focused on the CBME exhibition\n  - Involves client development and meeting sales targets\n  - Requires a Bachelor's degree and at least 1 year of sales experience\n\n- **Sales Executive** (Bangkok, Thailand) - Taylor and Francis\n  - Home-based position with travel requirements\n  - Promotes online resources to academic markets\n  - Requires 3+ years sales experience and strong English skills\n\n## Specialized Sales Positions\n- **Business Analyst - Sales** (Shawnee, KS)\n  -

In [5]:
import psycopg2
import pandas as pd

# Connection settings
DB_HOST = "postgres.dev.iris.informa.com"
DB_PORT = "5432"
DB_NAME = "aidb"
DB_USER = "v_svc_usr_aidb"
DB_PASSWORD = "j<pW@qNsFIc!(OR"

# Create connection
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    print("✅ Connected to aidb database")

    # Query langchain_pg_collection
    query_col = "select * from ai.langchain_pg_embedding  where collection_id IN (select uuid from ai.langchain_pg_collection where name = 'internal_private_employee_profiles_vectorstore');"
    df_col = pd.read_sql(query_col, conn)
    print("\nlangchain_pg_collection sample:")
    print(df_col)

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    if 'conn' in locals():
        conn.close()
        print("🔒 Connection closed")

✅ Connected to aidb database


C:\Users\SinhaK\AppData\Local\Temp\ipykernel_25840\1580065137.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_col = pd.read_sql(query_col, conn)



langchain_pg_collection sample:
                             collection_id  \
0     e2ea7251-7d14-421e-83b4-51c8b224f00c   
1     e2ea7251-7d14-421e-83b4-51c8b224f00c   
2     e2ea7251-7d14-421e-83b4-51c8b224f00c   
3     e2ea7251-7d14-421e-83b4-51c8b224f00c   
4     e2ea7251-7d14-421e-83b4-51c8b224f00c   
...                                    ...   
1400  e2ea7251-7d14-421e-83b4-51c8b224f00c   
1401  e2ea7251-7d14-421e-83b4-51c8b224f00c   
1402  e2ea7251-7d14-421e-83b4-51c8b224f00c   
1403  e2ea7251-7d14-421e-83b4-51c8b224f00c   
1404  e2ea7251-7d14-421e-83b4-51c8b224f00c   

                                              embedding  \
0     [-0.055500098,0.06580695,-0.017995978,-0.03934...   
1     [-0.042999007,0.044041406,-0.033617407,-0.0204...   
2     [-0.033228796,0.053677283,0.055665333,0.014910...   
3     [-0.04324236,0.0514435,0.049703863,0.034047145...   
4     [-0.088469006,0.04181069,-0.03438778,-0.029085...   
...                                                 ...   
1

In [23]:
import sys
print(sys.executable)
print("\nPYTHONPATH:")
for p in sys.path: 
    print(p)

try:
    import numpy as np
    print("\nNumPy:", np.__version__, " at ", np.__file__)
except Exception as e:
    print("NumPy import failed:", e)


c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\python.exe

PYTHONPATH:
c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\python311.zip
c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\DLLs
c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\Lib
c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env

c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\Lib\site-packages
c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\Lib\site-packages\win32
c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\Lib\site-packages\win32\lib
c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\Lib\site-packages\Pythonwin

NumPy: 2.3.2  at  c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\Lib\site-packages\numpy\__init__.py


In [1]:
import numpy as np
print(np.__version__, np.__file__)

1.26.4 c:\Users\SinhaK\AppData\Local\miniconda3\envs\elysia-dev-env\Lib\site-packages\numpy\__init__.py


In [ ]:
import psycopg2

# Database connection settings
DB_HOST = "localhost"        # Change if remote
DB_PORT = "5432"
DB_NAME = "aidb"
DB_USER = "your_username"
DB_PASSWORD = "your_password"

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    print("✅ Connected to PostgreSQL database!")

    # Create cursor
    cur = conn.cursor()

    # Query the employee_profile table
    cur.execute("SELECT * FROM employee_profile LIMIT 10;")
    rows = cur.fetchall()

    for row in rows:
        print(row)

    # Close connection
    cur.close()
    conn.close()
    print("✅ Connection closed.")

except Exception as e:
    print("❌ Error connecting to database:", e)



ModuleNotFoundError: No module named 'nltk'